# Class 10 - HTML to SQL, Part 2

In [1]:
from bs4 import BeautifulSoup

In [3]:
raw_html = open("menupages-morningside-heights.html").read() # read in as a string
soup = BeautifulSoup(raw_html, "html.parser") # parse it as html

In [4]:
search_table = soup.find("table", {"class": "search-results"})
table_body = search_table.find("tbody")

In [5]:
def get_name(tr_tag):
    name_address_tag = tr_tag.find("td", {"class": "name-address"})
    a_tag = name_address_tag.find("a")
    restaurant_name = a_tag.string
    return restaurant_name

def get_price(tr_tag):
    price_tag = tr_tag.find("td", {"class": "price"})
    price_span_tag = price_tag.find("span")
    if price_span_tag:
        price = int(price_span_tag.string)
    else:
        price = 0 
    return price

def get_cuisines(tr_tag):
    all_td_tags = tr_tag.find_all("td")
    cuisine_tag = all_td_tags[4]
    cuisines = cuisine_tag.string
    
    if cuisines:
        cuisines_list = cuisines.split(", ")
    else:
        cuisines_list = []
    return cuisines_list

In [6]:
restaurants = []
for tr_tag in table_body.find_all('tr'):
    
    restaurant_name = get_name(tr_tag)
    price = get_price(tr_tag)
    cuisines = get_cuisines(tr_tag)
    
    rest_dict = {'name': restaurant_name, 'price': int(price), 'cuisines': cuisines}
    restaurants.append(rest_dict)
    
restaurants

[{'cuisines': ['deli', 'sandwiches'], 'name': 'Milano Market', 'price': 2},
 {'cuisines': ['ethiopian', 'african'], 'name': 'Massawa', 'price': 0},
 {'cuisines': ['chinese', 'japanese', 'sushi'],
  'name': 'China Place',
  'price': 0},
 {'cuisines': ['cheese-steaks', 'deli', 'sandwiches', 'salads'],
  'name': 'Subsconscious',
  'price': 0},
 {'cuisines': ['italian', 'pizza'], 'name': 'Famous Famiglia', 'price': 0},
 {'cuisines': ['bakery-pastries', 'american', 'desserts'],
  'name': 'Kitchenette',
  'price': 0},
 {'cuisines': ['italian', 'pizza'], 'name': 'V & T Pizza', 'price': 0},
 {'cuisines': ['chinese'], 'name': 'New Aroma', 'price': 1},
 {'cuisines': ['chinese'], 'name': 'Peking Garden', 'price': 1},
 {'cuisines': ['diner'], 'name': "Tom's Restaurant", 'price': 1},
 {'cuisines': ['italian'], 'name': 'Pisticci', 'price': 1},
 {'cuisines': ['diner', 'american'], 'name': 'Deluxe', 'price': 1},
 {'cuisines': ['american', 'bar-food'], 'name': 'Toast', 'price': 2},
 {'cuisines': ['ital

In [7]:
import pandas as pd
df = pd.DataFrame(restaurants)

In [8]:
df

,cuisines,name,price
0,"[deli, sandwiches]",Milano Market,2
1,"[ethiopian, african]",Massawa,0
2,"[chinese, japanese, sushi]",China Place,0
3,"[cheese-steaks, deli, sandwiches, salads]",Subsconscious,0
4,"[italian, pizza]",Famous Famiglia,0
5,"[bakery-pastries, american, desserts]",Kitchenette,0
6,"[italian, pizza]",V & T Pizza,0
7,[chinese],New Aroma,1
8,[chinese],Peking Garden,1
9,[diner],Tom's Restaurant,1


Start thinking about how to structure your data so that it's easier to work with. 

# Putting data into SQL

We're going to have to start thinking about how to structure our data so that it's easier to work with! 

## Desigining our table schema

"schema" --> table designs

+ what table(s) do we need? 
+ what should those tables have in them (columns and data types)? 

**Why do we need more than one table?**
+ There are many-to-one and many-to-many relationships! 
+ Hint from the data: any one kind of cuisine have more than one restaurant associated with it
+ **Data normalization**: separating out the data so that any one piece of data only occurs in once place. 
    + Related term: **normal formal** 
    
What are our entities? 
+ Restaurants, each of which has a name, price, list of cuisines
+ which cuisines a restaurant is associated with (this isn't really a *thing*, it's relationship between two things

**PROPOSED:**
+ Restaurant table: 
    + `id` (unique integer identifying the restaurant)
    + `name` (string with restaurant's name)
    + `price`: (an integer that corresponds to the number of dollar signs
+ Cuisine table:
    + restaurant_id (number associated with restaurant)
    + kind (string that identifies the cuisine type itself)


**Sample entries in each table:**

```
sample entry from restaurant table:
id: 4
name: Brad's
price: 1
```

```
restaurant_id: 4
kind: seafood
```

**Sample query we want to be able to run:**

    SELECT restaurant.restaurant_id
    from restuarant JOIN cuisine ON restaurant.restaurant_id = cuisine.restaurant_id

Why name id and restaurant_id differently? It's a stylistic choice! 

## Setting up our database

+ **"Setup Phase" --> creating database, creating tables, "one-time" stuff --> happens in psql**
+ **"Working with data phase" --> inserting records, selecting stuff -> python**


### 1. Create the database in psql.

        CREATE DATABASE menupages;
        \c menupages
        \d : no relations found 

We need to create a unique integer id as our key. Solutions: number them 0-n, use the index, or our special SQL syntax! 

We've seen SQL data types before:
+ int integer
+ varchar(n) string with length n
+ numeric number/decimal
+ There is also a data type called **serial**, an integer that is automatically assigned.

### 2. Creating tables in psql

        CREATE TABLE restaurant(id serial, name varchar(80), price int);
        
**NOTE:** sequence table is how Postgres keeps track of the unique ids.

### 3. Inserting Data
        
        INSERT INTO restaurant(name, price) VALUES('Test Restaurant', 5);
        SELECT * from restaurant;
        INSERT INTO restaurant(name, price) VALUES('Test Cafe', 3);

### 4. Removing Data

        DELETE FROM restaurant;
        INSERT INTO restaurant(name, price) VALUES('Actual Restaurant', 1);
        DELETE FROM restaurant WHERE id = 3;
        
        
**NOTE:** Automatically incremented unique IDs don't know about deleted data! It will provide you the next integer, given the table's entire history.
        
### 5. Adding our cuisine table

Remember that restaurant id is not unique -- it's just the id value from the other table. 

        CREATE TABLE cuisine(restaurant_id int, kind varchar(40)); 
        \d cuisine
        
### 6. Importing data from one restaurant

        INSERT INTO restaurant(name, price) VALUES ('Panino Sportivo Roma', 2);

To insert into our cuisines table, we could just look at the id and cuisines listed in our restaurant table and add that data"manually" into table. 

        INSERT INTO cuisine (restaurant_id, kind) VALUES (4, 'italian')
        INSERT INTO cuisine (restaurant_id, kind) VALUES (4, 'coffee-tea')
        INSERT INTO cuisine (restaurant_id, kind) VALUES (4, 'sandwiches')

        INSERT INTO restaurant (name, price) VALUES ('Max Soha', 2) RETURNING id;

This allows us to see the unique id right away and use it for the cuisines table. 

        INSERT INTO cuisine (restaurant_id, kind) VALUES (5, 'italian')

## We're going to be writing our insert and select statements inside python! 

In [28]:
import pg8000

In [58]:
conn = pg8000.connect(database="menupages")

In [59]:
type(conn)

pg8000.core.Connection

In [67]:
# For our troubleshooting later
conn.rollback()

### The cursor object

The cursor object is the thing that you issue queries through. It allows you to issue statements and get things back from those statements! 

        .execute() # executes a SQL statement
        .fetchone() # fetches the first record of the results of a statement (as a list)
        .fetchall() # returns all the rows of the resaults of a statement as a list of lists

In [111]:
# The cursor object is the thing that you issue queries through
# allows you to issue statements
# and get things back from those statements
cursor = conn.cursor()

In [62]:
# Telling python: here's a string I would like to execute in SQL
# You don't need to specify a semi-colon
cursor.execute("INSERT INTO restaurant (name, price) VALUES ('Good Food Place', 3)")
conn.commit()

In [63]:
cursor.execute("SELECT * FROM restaurant")
for item in cursor.fetchall():
    print(item)

[1, 'Good Food Place', 3]
[2, 'Panino Sportivo Roma', 2]
[3, 'Good Food Place', 3]
[10, 'Good Food Place', 3]


In [68]:
cursor.execute("INSERT INTO restaurant (name, price) VALUES ('Palace of Vegan Nosh', 3) RETURNING id")
results = cursor.fetchone()
conn.commit()

In [69]:
results

[13]

In [70]:
rowid = results[0]
rowid

13

In [71]:
cursor.execute("INSERT INTO restaurant (name, price) VALUES ('Another restaurant', 2) RETURNING id")
rowid = cursor.fetchone()[0]
conn.commit()

In [72]:
rowid

14

### Issues with quoting strings & parameters

In [ ]:
# SQL is going to get to the ' in Brad's, and freak out! We need an escape character! 
cursor.execute("INSERT INTO restaurant (name, price) VALUES ('Brad's', 2) RETURNING id")
rowid = cursor.fetchone()[0]
conn.commit

In [ ]:
# SQL injection attack
restaurant = cursor.execute("'Restaurant'); DELETE FROM restaurant;")


string in python -> "quote" "escape" -> valid SQL statement

Don't write what we wrote above!

This is bad:

        cursor.execute("INSERT INTO restaurant (name, price) VALUES ('Brad's', 2) RETURNING id")

In [73]:
conn.rollback()
rest_insert = "INSERT INTO restaurant (name, price) VALUES (%s, %s)"
cursor.execute(rest_insert, ["Brad's", 1])

#pg8000 will do the work of plugging the values from our second line into %s, %s
# An aside: double single quotes such as '' would also solve our Brad's problem 
# (if we were running that from python, write \'\')
conn.commit()

In [74]:
cursor.execute("INSERT INTO restaurant (name, price) VALUES (%s, %s) RETURNING id", ["Test Restaurant", 2])

rowid = cursor.fetchone()[0]
conn.commit()

In [75]:
rowid

16

In [112]:
cuisine_insert = "INSERT INTO cuisine (restaurant_id, kind) VALUES (%s, %s)"
cursor.execute(cuisine_insert, [rowid, "fondue"])
cursor.execute(cuisine_insert, [rowid, "casseroles"])
conn.commit()


In [78]:
restaurants

[{'cuisines': ['deli', 'sandwiches'], 'name': 'Milano Market', 'price': 2},
 {'cuisines': ['ethiopian', 'african'], 'name': 'Massawa', 'price': 0},
 {'cuisines': ['chinese', 'japanese', 'sushi'],
  'name': 'China Place',
  'price': 0},
 {'cuisines': ['cheese-steaks', 'deli', 'sandwiches', 'salads'],
  'name': 'Subsconscious',
  'price': 0},
 {'cuisines': ['italian', 'pizza'], 'name': 'Famous Famiglia', 'price': 0},
 {'cuisines': ['bakery-pastries', 'american', 'desserts'],
  'name': 'Kitchenette',
  'price': 0},
 {'cuisines': ['italian', 'pizza'], 'name': 'V & T Pizza', 'price': 0},
 {'cuisines': ['chinese'], 'name': 'New Aroma', 'price': 1},
 {'cuisines': ['chinese'], 'name': 'Peking Garden', 'price': 1},
 {'cuisines': ['diner'], 'name': "Tom's Restaurant", 'price': 1},
 {'cuisines': ['italian'], 'name': 'Pisticci', 'price': 1},
 {'cuisines': ['diner', 'american'], 'name': 'Deluxe', 'price': 1},
 {'cuisines': ['american', 'bar-food'], 'name': 'Toast', 'price': 2},
 {'cuisines': ['ital

## Inserting many restaurants

In [ ]:
rest_insert = "INSERT INTO restaurant (name, price) VALUES (%s, %s)"

for item in restaurants: 
    cursor.execute(rest_insert, [item['name'], item['price']])

conn.commit()

# If we run this cell we get an error:
# <bs4.element.NavigableString> not mapped to pg type

In [80]:
# let's look at our data
first = restaurants[0]
first

{'cuisines': ['deli', 'sandwiches'], 'name': 'Milano Market', 'price': 2}

In [81]:
print(first['name'])

Milano Market


In [83]:
print(type(first['name']))

# Oh no, we wanted a regular ol' string! 
# Whenever you use the .string attribute of a Beautiful Soup tag object, 
# the type of that value is `bs4.element.NavigableString`

# fortunately, we have an easy fix: str(val)

<class 'bs4.element.NavigableString'>


In [113]:
rest_insert = "insert into restaurant (name, price) values (%s, %s)"

for item in restaurants:
    cursor.execute(rest_insert, [str(item['name']), item['price']])
conn.commit()

## Inserting both restaurants AND their cuisines

Delete everything from our postgres database! In psql:
+ `delete from restaurant;`
+ `delete from cuisine`

In [114]:
rest_insert = "insert into restaurant (name, price) values (%s, %s) returning id"
cuisine_insert = "insert into cuisine (restaurant_id, kind) values (%s, %s)"

for item in restaurants:
    
    print("inserting restaurant", item['name'])
    
    # insert restaurant, RETURNING id
    cursor.execute(rest_insert, [ str(item['name']), item['price'] ])
    rowid = cursor.fetchone()[0]
    
    for cuisine in item['cuisines']:
        
        # insert restaurant_id, cuisine kind into cuisine table
        print("inserting cuisine", cuisine)
        cursor.execute(cuisine_insert, [rowid, str(cuisine)])        
        
conn.commit()

inserting restaurant Milano Market
inserting cuisine deli
inserting cuisine sandwiches
inserting restaurant Massawa
inserting cuisine ethiopian
inserting cuisine african
inserting restaurant China Place
inserting cuisine chinese
inserting cuisine japanese
inserting cuisine sushi
inserting restaurant Subsconscious
inserting cuisine cheese-steaks
inserting cuisine deli
inserting cuisine sandwiches
inserting cuisine salads
inserting restaurant Famous Famiglia
inserting cuisine italian
inserting cuisine pizza
inserting restaurant Kitchenette
inserting cuisine bakery-pastries
inserting cuisine american
inserting cuisine desserts
inserting restaurant V & T Pizza
inserting cuisine italian
inserting cuisine pizza
inserting restaurant New Aroma
inserting cuisine chinese
inserting restaurant Peking Garden
inserting cuisine chinese
inserting restaurant Tom's Restaurant
inserting cuisine diner
inserting restaurant Pisticci
inserting cuisine italian
inserting restaurant Deluxe
inserting cuisine din

### An aside:

In [104]:
"""use triple quotes
to continue strings
on multiple lines"""

'use triple quotes\nto continue strings\non multiple lines'